In [1]:
import numpy as np

import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.2.0
Eager mode:  True
Hub version:  0.8.0
GPU is NOT AVAILABLE


In [2]:
train_data,validation_data,test_data=tfds.load(        
            name='imdb_reviews',
            split=('train[:60%]','train[60%:]','test'),
            as_supervised=True
)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteZJOATX/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteZJOATX/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteZJOATX/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [3]:
train_examples_batch,train_labels_batch=next(iter(train_data.batch(10)))
print(train_examples_batch)

tf.Tensor(
[b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
 b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot de

In [4]:
print(train_labels_batch)

tf.Tensor([0 0 0 1 1 1 0 0 0 0], shape=(10,), dtype=int64)


In [5]:
embedding='https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1'

hub_layer=hub.KerasLayer(embedding,input_shape=[],dtype=tf.string,trainable =True)

print(hub_layer(train_examples_batch[:3]))

tf.Tensor(
[[ 1.765786   -3.882232    3.9134233  -1.5557289  -3.3362343  -1.7357955
  -1.9954445   1.2989551   5.081598   -1.1041286  -2.0503852  -0.72675157
  -0.65675956  0.24436149 -3.7208383   2.0954835   2.2969332  -2.0689783
  -2.9489717  -1.1315987 ]
 [ 1.8804485  -2.5852382   3.4066997   1.0982676  -4.056685   -4.891284
  -2.785554    1.3874227   3.8476458  -0.9256538  -1.896706    1.2113281
   0.11474707  0.76209456 -4.8791065   2.906149    4.7087674  -2.3652055
  -3.5015898  -1.6390051 ]
 [ 0.71152234 -0.6353217   1.7385626  -1.1168286  -0.5451594  -1.1808156
   0.09504455  1.4653089   0.66059524  0.79308075 -2.2268345   0.07446612
  -1.4075904  -0.70645386 -1.907037    1.4419787   1.9551861  -0.42660055
  -2.8022065   0.43727064]], shape=(3, 20), dtype=float32)


In [6]:
model=tf.keras.Sequential([
    
    hub_layer,
    tf.keras.layers.Dense(16,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')

])


model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [7]:
history=model.fit(
    train_data.shuffle(10000).batch(512),
    epochs=20,
    validation_data=validation_data.batch(512),
    verbose=1
)

Epoch 1/20
30/30 [==============================] - 4s 145ms/step - loss: 0.8251 - accuracy: 0.4717 - val_loss: 0.7561 - val_accuracy: 0.4550
Epoch 2/20
30/30 [==============================] - 4s 140ms/step - loss: 0.7191 - accuracy: 0.5117 - val_loss: 0.6920 - val_accuracy: 0.5482
Epoch 3/20
30/30 [==============================] - 4s 143ms/step - loss: 0.6609 - accuracy: 0.6062 - val_loss: 0.6369 - val_accuracy: 0.6389
Epoch 4/20
30/30 [==============================] - 4s 140ms/step - loss: 0.6060 - accuracy: 0.6713 - val_loss: 0.5806 - val_accuracy: 0.7032
Epoch 5/20
30/30 [==============================] - 4s 117ms/step - loss: 0.5585 - accuracy: 0.7170 - val_loss: 0.5443 - val_accuracy: 0.7328
Epoch 6/20
30/30 [==============================] - 4s 119ms/step - loss: 0.5172 - accuracy: 0.7532 - val_loss: 0.5089 - val_accuracy: 0.7587
Epoch 7/20
30/30 [==============================] - 3s 113ms/step - loss: 0.4757 - accuracy: 0.7845 - val_loss: 0.4722 - val_accuracy: 0.7851
Epoch 

In [8]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))

49/49 - 3s - loss: 0.3169 - accuracy: 0.8658
loss: 0.317
accuracy: 0.866


In [9]:
sentence=['you are really amazing!!!']#enter a sentence here 
classes=model.predict(sentence)
if classes > 0.5:
    print('Positive')
else:
    print('Negative')

Positive
